# Bond and Currency Swap Analysis

This notebook analyzes bond returns and currency swap strategies using yield curve data.

**Author:** Brett Cooper  
**Date:** 2025-01-15  
**Purpose:** Replicate and extend the R analysis in Python with improved reproducibility

## Openscapes Principles Applied
- **Reproducible:** All dependencies specified in environment.yml
- **Shareable:** Clear documentation and modular code
- **Data-driven:** Transparent data loading and validation
- **Collaborative:** Well-documented functions and analysis steps

In [ ]:
# Import required libraries
import sys
sys.path.insert(0, '../src')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Import our custom modules
from yield_curves.helpers import round_month_to_maturity
from yield_curves.swap_analysis import SwapAnalysis

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Display options
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)

print("Libraries loaded successfully!")

## 1. Load Data

Load the yield curve data and FX exchange rates.

In [ ]:
# Define data paths
DATA_DIR = Path('../data/raw')

# Load exchange rate data
fx_data = pd.read_csv(DATA_DIR / 'wpu_exchange_rates.csv', parse_dates=['Date'])
fx_data = fx_data.set_index('Date')

print(f"FX Data shape: {fx_data.shape}")
print(f"Date range: {fx_data.index.min()} to {fx_data.index.max()}")
fx_data.head()

In [ ]:
# Load fitted yield curve data
# Note: This would come from the yield curve fitting process
# For now, we'll show the structure

# This would be loaded from the .Rdata equivalent or regenerated
print("Yield curve data would be loaded here from processed files")
print("Structure: 3D array with dimensions [country, date, maturity]")

## 2. Set Up Swap Analysis

Initialize the swap analysis with bond parameters.

In [ ]:
# Define parameters
ORIG_DATE = '2018-12-31'
MAT_DATE = '2026-12-31'
NUM_DIG_MATURITY = 4

# Initialize swap analysis
swap = SwapAnalysis(
    origination_date=ORIG_DATE,
    maturity_date=MAT_DATE,
    num_digits_maturity=NUM_DIG_MATURITY
)

print(f"Analysis period: {swap.orig_date} to {swap.mat_date}")
print(f"Number of months: {len(swap.results)}")
swap.results.head()

## 3. Calculate Currency Returns

Compute monthly FX returns for the WPU currency basket.

In [ ]:
# Calculate WPU/USD returns
# First, get month-end values
month_ends = swap.results.index

# Merge and forward-fill FX data to month-ends
fx_aligned = fx_data.reindex(
    fx_data.index.union(month_ends)
).fillna(method='ffill').loc[month_ends]

# Calculate returns
wpu_returns = fx_aligned['WPUUSD'].pct_change()

# Set currency returns in swap analysis
swap.set_currency_returns(
    long_ccy_returns=wpu_returns,
    short_ccy_returns=None  # USD is base currency
)

print("Currency returns calculated")
print(f"Mean monthly return: {wpu_returns.mean():.4%}")
print(f"Annualized volatility: {wpu_returns.std() * np.sqrt(12):.2%}")

## 4. Visualize FX Returns

Plot the WPU/USD exchange rate and returns over time.

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Plot exchange rate
axes[0].plot(fx_aligned.index, fx_aligned['WPUUSD'], linewidth=2)
axes[0].set_title('WPU/USD Exchange Rate', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Exchange Rate')
axes[0].grid(True, alpha=0.3)

# Plot returns
axes[1].plot(wpu_returns.index, wpu_returns * 100, linewidth=1.5)
axes[1].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[1].set_title('Monthly WPU/USD Returns', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Return (%)')
axes[1].set_xlabel('Date')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Calculate Bond Returns and Swap Performance

This section would calculate returns based on yield curve data.

**Note:** Requires yield curve data to be loaded first.

In [ ]:
# Placeholder for yield data setting
# swap.set_yields(long_yields_df, short_yields_df)
# swap.calculate_returns()

print("This section requires fitted yield curve data.")
print("See notebook 02_yield_curve_fitting.ipynb for yield curve construction.")

## 6. Performance Summary

Calculate and display performance metrics when data is available.

In [ ]:
# performance = swap.get_performance_summary()
# print("\nPerformance Summary:")
# print(performance.round(4))

## Next Steps

1. Run `02_yield_curve_fitting.ipynb` to generate fitted yield curves
2. Return to this notebook to complete the swap analysis
3. Explore different currency pairs and maturity strategies

## References

- Henckel bond return formula documentation
- WPU currency basket methodology
- Data sources: [Add your data sources here]